# Make bilby samples from a list of galaxy hosts

This is a script to make bilby samples from a list of galaxy hosts. 

- The list of galaxy hosts is provided in a csv
- Galaxies are ingested, and the following quantities are propagated to the bilby sample
    - ra
    - dec
    - dL prior needs to be updated for the SkySim cosmology

In [1]:
import bilby as bb
import numpy as np
import gwcosmo as gwc
import pandas as pd
import GCR
import GCRCatalogs as GCRCat
import matplotlib.pyplot as plt
import os
from astropy.time import Time

/pscratch/sd/s/seanmacb/gwCosmoDesc/lib/python3.10/site-packages/gwcosmo/utilities/cosmology.py:20: UserWarning: Wswiglal-redir-stdio:

SWIGLAL standard output/error redirection is enabled in IPython.
This may lead to performance penalties. To disable locally, use:

with lal.no_swig_redirect_standard_output_error():
    ...

To disable globally, use:

lal.swig_redirect_standard_output_error(False)

Note however that this will likely lead to error messages from
LAL functions being either misdirected or lost when called from
Jupyter notebooks.

To suppress this warning, use:

import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
import lal

  import lal


In [2]:
cat_name2 = "skysim5000_v1.2"
skysimCat = GCRCat.load_catalog(cat_name2) # Load the skysim catalog
hostDF = pd.read_csv("/global/homes/s/seanmacb/DESC/DESC-GW/gwStreetlights/data/mergers-w=Lum,n=1e7,FromSkySim50.csv") # Load the CBC catalog

In [3]:
prior = bb.gw.prior.BBHPriorDict(aligned_spin=False) # The bbh prior, spins misaligned
prior["luminosity_distance"] = bb.gw.prior.UniformSourceFrame(0,5000,cosmology=skysimCat.cosmology,name='luminosity_distance', latex_label='$d_L$', unit='Mpc', boundary=None) # Update the luminosity distance prior, based on 

13:06 bilby INFO    : No prior given, using default BBH priors in /pscratch/sd/s/seanmacb/gwCosmoDesc/lib/python3.10/site-packages/bilby/gw/prior_files/precessing_spins_bbh.prior.


In [4]:
prior.sample()

{'mass_ratio': 0.2197770643071324,
 'chirp_mass': 28.194144030246054,
 'luminosity_distance': 2513.562546230264,
 'dec': 0.21090874217923028,
 'ra': 1.5356516678050518,
 'theta_jn': 0.9551834466207759,
 'psi': 0.21308699369978906,
 'phase': 3.0195503477138432,
 'a_1': 0.8644170745950256,
 'a_2': 0.28623189766616786,
 'tilt_1': 1.6804615961417257,
 'tilt_2': 0.8626706137756249,
 'phi_12': 5.78585803356507,
 'phi_jl': 5.351892858379443}

In [5]:
skysimCat.cosmology.luminosity_distance(hostDF.loc[0]["redshiftHubble"])

<Quantity 23772.32428394 Mpc>

## Let's do the sampling this way
- For each entry in the CBC catalog
    - take a sample of the bilby prior, most parameters will be added
    - For ra/dec, use mra and mdec from the cbc catalog
    - For dL, just compute directly based on the cosmology of SkySim
- Add to the dataframe
- Save the dataframe to disk

In [6]:
keys = list(prior.sample())
keys.pop(3)
keys.pop(3)

'ra'

In [ ]:
injDict = {}
for k in keys:
    injDict[k] = []

cnt = 0
for ids,row in hostDF.iterrows():
    thisSample = prior.sample()
    for k in keys:
        if k!="luminosity_distance":
            injDict[k].append(thisSample[k])
    injDict["luminosity_distance"].append(float(str(skysimCat.cosmology.luminosity_distance(row["redshiftHubble"])).split(" ")[0]))
    if cnt % 100000 == 0:
        print("{}% finished".format(cnt//100000))
    cnt+=1

0% finished


In [ ]:
# for v in injDict.values():
#     print(np.shape(v))

In [ ]:
for k in injDict.keys():
    hostDF[k] = injDict[k]

In [ ]:
float(str(skysimCat.cosmology.luminosity_distance(row["redshiftHubble"])).split(" ")[0])

## Save this

In [ ]:
saveColumns = hostDF.columns.values[1:] # This is because of the extra column - check this always!!!

In [ ]:
dataDir = "/global/homes/s/seanmacb/DESC/DESC-GW/gwStreetlights/data"
hostDF.to_csv(os.path.join(dataDir,"mergers-w=Lum,n=1e7,FromSkySim50_withBilby.csv"),columns=saveColumns,index=False)

## OOPS! Add a column to denote if the CBC has been sampled before

In [ ]:
hostDF = pd.read_csv(os.path.join(dataDir,"mergers-w=Lum,n=1e7,FromSkySim50_withBilby.csv"))

In [ ]:
hostDF["sampled"] = False

In [ ]:
hostDF.to_csv(os.path.join(dataDir,"mergers-w=Lum,n=1e7,FromSkySim50_withBilby.csv"),columns=saveColumns,index=False)